In [ ]:
h_p_meas = 0.18/100
h_p_reset = 0.18/100
h_p_gate1 = 0.4/100
h_p_gate2 = 2.7/100

# thermal relaxation error (nanosecond time)
h_T1s = random.sample(range(10, 100, 1), 10) # consider adjusting range to be more reasonable
h_T2 = 1e9

h_time_meas = random.sample(range(600, 700, 1), 10) # assumed same as aria
h_time_u1 = 0 # virtual gate?
h_time_gate1 = 10e3 # u1 and u2
h_time_gate2 = 210e3 # 2 qubit gate

# thermal relaxation error
h_ther_u1  = [thermal_relaxation_error(t1 * 1e9, h_T2, h_time_u1)
              for t1 in h_T1s]
h_ther_gate1  = [thermal_relaxation_error(t1 * 1e9, h_T2, h_time_gate1)
              for t1 in h_T1s]
h_ther_gate2 = [[thermal_relaxation_error(t1a * 1e9, h_T2, h_time_gate2).expand(
             thermal_relaxation_error(t1b * 1e9, h_T2, h_time_gate2))
              for t1a in h_T1s]
               for t1b in h_T1s]

# depolarizing error
h_dep_reset = depolarizing_error(h_p_reset, 1)
h_dep_gate1 = depolarizing_error(h_p_gate1 - 9.99995e-6, 1)
h_dep_gate2 = depolarizing_error(h_p_gate2 - 2.09977952e-4, 2)

# readout error
h_er_meas = ReadoutError([[1 - p_meas, p_meas], [p_meas, 1 - p_meas]])

fake_harm = NoiseModel()

for j in range(10):
    fake_harm.add_quantum_error(h_ther_u1[j].compose(h_dep_gate1), "u1", [j])
    fake_harm.add_quantum_error(h_ther_gate1[j].compose(h_dep_gate1), "u2", [j])
    fake_harm.add_quantum_error(h_ther_gate1[j].compose(h_dep_gate1), "u3", [j])
    for k in range(10):
        fake_harm.add_quantum_error(h_ther_gate2[j][k].compose(h_dep_gate2), "cx", [j, k])
fake_harm.add_all_qubit_quantum_error(h_dep_reset, "reset")
fake_harm.add_all_qubit_readout_error(h_er_meas)